![uc3m](img/uc3m.jpg)

# Optimization and Analytics
## Bachelor in Data Science and Engineering 
## First homework: Linear and Discrete Models (Topics 1,2)


Dimitar Ilev

 ### General Objectives

 - Find and describe a realistic problem where the decision making process requires of the formulation
of a linear optimization model
 - The problem must have at least 20 decision variables and more than 3 non-trivial constraints
 - The bigger the problem, the better

 #### Evaluation 
 
 a) (2 points) Formulate the problem as a linear optimization model (general formulation). Identify the
model sets, parameters, variables, objective function and constraints.

b) (3 points) Implement the model in a notebook (preferably using Pyomo, but not mandatory) and
solve it for an input based on real (or realistic) world data. Interpret the solution.

c) (1 point) Compute the sensitivities associated with each constraint, and interpret their values.

d) (4 points) Modify the problem in a) to impose some logical and conditional constraints that require
the use of binary or integer variables. The more integer variables or constraints, the better.
Implement and solve this new model and interpret the results.

Remember the deadline: November 3, 2022 at 23:00

IMPORTANT: Present the code in a Jupyter notebook and name it as “Surname-Name-HW1.ipynb”.
Upload also the corresponding complied version “Surname-Name-HW1.html”, and the datasets that are
needed (if any) to reproduce your results.

## Idea
- Maximising the profit of a resale business for industrial electronis
### Intro
- Peter is a software developer and in his free time he likes to buy and sell tech producs on the resell market in small manegable quantities. He is considerring to open up his own business and wants to find out how he can maximize the profits of his investments. Therefore, he is in desperate need of a model, that can help him figure it out.
### Description
- For the solution of Peter's problem the implementation of a linear and discrete algorithm will be used, thus allowing the evaluation of his current financial and economical situation, for determining whether he should start his own business or not.
### Steps
1. Decission variables
2. Parameters
3. Objectives
4. Constraints
### Use


## Statement

We have a set of factories I = [1,2] and set of demand markets J = [1,2,3]. Each market has some product demand $d_j$ and each factory has production capacity $M_i$. There are also transportation costs to deliver one unit from factory i to market j. The goal is to satisfy the demand with minimum cost.

  Distance      | M1 | M2 | M3 | offer
  --------------|-----|---|----|--
  F1  | 2.5 | 1.7 | 1.8 | 350
  F2  | 2.5 | 1.7 | 1.8 | 350
  demand  | 325 | 300 | 275 |

### Import of the Pyomo module

In [5]:
from pyomo.opt import SolverFactory
import sys
from pyomo.environ import *  # import environment
import pyomo.environ as pyo

solvername='glpk'
solverpath_folder='C:\\w64'
solverpath_exe='C:\\w64\\glpsol'

sys.path.append(solverpath_folder)
solver=SolverFactory(solvername, executable=solverpath_exe)

### Optimization framework

Input (data) $\rightarrow$ Model (var, obj, const) $\rightarrow$ Output (solution)

### Define problem data: sets and parameters

Sets are created as attributes object of the main model objects 

Parameter objects are created specifying the sets over which they are defined and are initialised with either a python dictionary or a scalar

In [6]:
# Import data set
import pandas as pd
mydata = pd.read_csv("C:\\BSc in EIB\\5. Semester\\Repos\Repo - Optimization and Analytics\\dt1.csv")

print(mydata)

    VFD-Type Nr.  Sell-Price  Material cost  Net-Profit  \
0           1503          35             13          22   
1           1462          98             14          84   
2           1168          57              7          50   
3           1612          27              8          19   
4           1280          30             12          18   
5           1088          74             12          62   
6           1414          32              9          23   
7           1235          64              5          59   
8           1301          96              8          88   
9           1581          93             15          78   
10          1587          53              8          45   
11          1440          51             14          37   
12          1503          37             10          27   
13          1993          90              9          81   
14          1776          22             12          10   
15          1703          65             15          50 

In [7]:
# The data

mTransport = ConcreteModel('Transportation Problem')
#Define i and j
mTransport.i = Set(initialize=['P1', 'P2'], doc='origins' )
mTransport.j = Set(initialize=['M1', 'M2', 'M3'], doc='destinations')

#Define capacity and demand
mTransport.Cap = Param(mTransport.i, initialize={'P1' : 350, 'P2': 600 }, doc='origin capacity' )
mTransport.Dem = Param(mTransport.j, initialize={'M1': 325, 'M2': 300, 'M3': 275}, doc='destination demand')

TransportationCost = {
    ('P1', 'M1' ): 2.5,
    ('P1', 'M2'): 1.7,
    ('P1', 'M3' ): 1.8,
    ('P2', 'M1' ): 2.5,
    ('P2', 'M2'): 1.8,
    ('P2', 'M3' ): 1.4
    }


mTransport.Dist = Param(mTransport.i, mTransport.j, initialize=TransportationCost, doc='Distance in hundreds of km')
mTransport.f = 90 # Fixed cost per 100 km in euros

#  Parameter c(i,j)  transport cost in thousands of dollars per case
#            c(i,j) = f * Dist(i,j) 

def cost_euro(mTransport, i, j):
  return mTransport.f * mTransport.Dist[i,j] 

mTransport.Cost = Param(mTransport.i, mTransport.j, initialize=cost_euro, doc='Transportation cost per 100 km in eur')

In [1]:
mTransport.pprint()

NameError: name 'mTransport' is not defined

### General formulation for the transportation problem

\begin{align*}
\underset{x_{ij}}{\min} & \quad \sum_{i=1}^{n}\sum_{j=1}^{m}c_{ij}x_{ij}\\
\text{s.t.:}&\\
  &\sum_{i=1}^{n} x_{ij} \geq D_{j} \quad \forall j\\
  &\sum_{j=1}^{m} x_{ij} \leq \bar{C}_{i} \quad \forall i\\
  &x_{ij} \geq 0 \quad \forall ij 
\end{align*}

### 1. Define the variables

In [9]:
# define x(i,j)
mTransport.x = Var(mTransport.i, mTransport.j, doc='units transported', within=NonNegativeReals)

### 2. Define the objective function

In [10]:
def objective_rule(mTransport):
  return sum(mTransport.Cost[i,j]*mTransport.x[i,j] for i in mTransport.i for j in mTransport.j)

mTransport.objective = Objective(rule=objective_rule, sense=minimize, doc='transportation cost')


### 3. Define the constraints

In [11]:
def Capacity(mTransport, i):
  return sum(mTransport.x[i,j] for j in mTransport.j) <= mTransport.Cap[i]

mTransport.Capacity = Constraint(mTransport.i, rule=Capacity, doc='capacity at origin')
    
def Demand (mTransport, j):
  return sum(mTransport.x[i,j] for i in mTransport.i) >= mTransport.Dem[j]

mTransport.Demand = Constraint(mTransport.j, rule=Demand, doc='demand at destination' )




### Let's see the solution

In [12]:
#Solver = pyo.SolverFactory(solvername,executable=solverpath_exe)#('glpk')
solvername='glpk'
solverpath_folder='C:\\w64'
solverpath_exe='C:\\w64\\glpsol'

sys.path.append(solverpath_folder)
solver=SolverFactory(solvername, executable=solverpath_exe)
Results = solver.solve(mTransport)

# Display solution
mTransport.x.display()


x : units transported
    Size=6, Index=x_index
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('P1', 'M1') :     0 :  50.0 :  None : False : False : NonNegativeReals
    ('P1', 'M2') :     0 : 300.0 :  None : False : False : NonNegativeReals
    ('P1', 'M3') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('P2', 'M1') :     0 : 275.0 :  None : False : False : NonNegativeReals
    ('P2', 'M2') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('P2', 'M3') :     0 : 275.0 :  None : False : False : NonNegativeReals


Display solution using pandas

In [13]:
import pandas as pd

transp_results = pd.DataFrame()

for i in mTransport.i:
    for j in mTransport.j:
        transp_results.loc[i,j] = mTransport.x[i,j]()
        
        
print(transp_results)

       M1     M2     M3
P1   50.0  300.0    0.0
P2  275.0    0.0  275.0


### Now the minimum cost

In [14]:
mTransport.objective()

153675.0

### Interpretation

In order to attain the minimum transport costs, 50 units should be sent from P1 to M1, 300 from P1 to M2, 275 from P2 to M1, and 275 from P2 to M3

The total transport costs will be 153675 euros

### We are done

![uc3m](img/wedidit.png)